## 小数据集

In [2]:
import numpy as np
import pandas as pd
import jieba
import csv
import sklearn

## 载入数据

In [3]:
data = open('Data/small/test.txt','r',encoding='utf-8')
f=open('Data/data_new/small.csv','a',newline='',encoding='utf-8')

csv_writer = csv.writer(f)

m=data.readlines()

csv_writer.writerow(["title","tag"])

for i in m:
    n=i.strip('\n')
    L=n.split('\t')
    # print(L)
    csv_writer.writerow(L)
    
f.close()
data.close()

In [3]:
data=pd.read_csv('Data/data_new/small.csv')
data.head()

,title,tag
0,今年一季度全国纪检监察机关处分省部级干部16人,0
1,习近平：携手迎接挑战 合作开创未来,0
2,瞭望丨引领人类海洋文明发展方向,0
3,广东省老区湾区同携手 接续奋进新时代,0
4,关注全民健身：健身场地少、消费高等难题怎么破？,0


## 划分训练集测试集

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['title'], data['tag'], random_state=1)



In [5]:
print(x_test.shape[0])
print(type(y_test))

601
<class 'pandas.core.series.Series'>


## 分词

In [6]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\32852\AppData\Local\Temp\jieba.cache
Loading model cost 0.601 seconds.
Prefix dict has been built successfully.


2034    泰达 5 - 1 送 恒丰 5 连败 ， 惠 家康 电摩 各 2 球 ， 买 提江 圆月 弯刀
1514        阿斯顿 · 马丁 AMR 谍 照曝光 ， 壕 无人性 ， 网友 ： 想想 就 可以 了
229         中国人民银行 决定 于 2022 年 4 月 25 日 下调 金融机构 存款 准备金率
1586             思域 ， 让 我 告诉 你 它 是 怎么 坑害 中国 消费者 的 （ 上 ）
2196              上联 ： 二八 少女 一朵花 ， 花开花落 知多少 ？ 怎么 对 下联 ？
Name: title, dtype: object

## 停词

In [7]:
infile = open("hit_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

## 向量化

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords, max_features=5000)
vectorizer.fit(x_train_fenci)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CountVectorizer(max_features=5000,
                stop_words=['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→',
                            '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一',
                            './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］',
                            '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', ...])

In [9]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
#模型训练
classifier.fit(vectorizer.transform(x_train_fenci), y_train)
#使用训练好的模型进行预测
classifier.score(vectorizer.transform(fenci(x_test)), y_test)

0.7038269550748752

## 预测

In [10]:
from sklearn.metrics import classification_report
predict=classifier.predict(vectorizer.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.60      0.62      0.61        56
           1       0.83      0.64      0.72        85
           2       0.73      0.83      0.78        65
           3       0.70      0.69      0.69        64
           4       0.77      0.80      0.79        71
           5       0.76      0.79      0.78        73
           6       0.58      0.77      0.66        56
           7       0.78      0.47      0.59        89
           8       0.57      0.86      0.69        42

    accuracy                           0.70       601
   macro avg       0.70      0.72      0.70       601
weighted avg       0.72      0.70      0.70       601



In [13]:
f=open('test_5.txt','r',encoding='utf-8')

L=[]
m=f.readlines()
for i in m:
    n=i.strip('\n')
    x=n.split('\t')
    L.append(x[1])

data_to_predict=pd.Series(L)
# data_to_predict=np.matrix(data_to_predict)

In [14]:
print(data_to_predict.shape)

(603,)


In [15]:
predict=classifier.predict(vectorizer.transform(fenci(data_to_predict)))

In [16]:
print(predict)

[5 1 6 2 3 1 7 4 2 7 7 0 4 7 3 4 5 5 1 3 0 4 7 7 7 3 6 1 3 4 1 0 2 4 0 5 4
 7 7 3 5 0 6 5 5 7 4 0 1 0 5 5 5 7 2 5 4 2 2 0 0 0 1 1 3 4 6 7 2 0 6 6 1 0
 0 0 8 5 5 3 4 4 6 7 2 0 4 3 3 0 2 3 2 0 4 0 3 5 3 5 5 5 7 4 4 3 3 7 1 0 7
 0 7 3 3 5 4 0 2 7 0 1 2 1 0 7 0 4 5 6 1 4 4 1 5 2 7 0 0 2 7 3 5 4 5 7 7 1
 0 0 4 1 5 3 0 1 0 7 0 1 7 6 0 5 2 0 1 6 4 1 8 7 6 6 1 5 0 3 2 0 4 0 4 3 1
 7 7 0 5 8 5 0 2 0 2 2 2 4 5 1 7 3 7 0 1 4 8 5 8 2 7 5 2 6 4 1 0 4 0 7 1 2
 5 4 3 2 0 4 3 7 6 4 4 1 0 7 6 7 3 3 4 7 2 0 4 4 5 0 2 5 7 2 5 3 7 6 7 6 7
 0 3 0 4 6 1 3 3 0 7 7 3 6 7 5 5 2 4 6 7 3 2 1 4 3 0 7 6 3 2 2 0 4 6 7 0 0
 8 8 7 1 3 8 7 0 8 5 0 8 5 4 7 1 1 5 3 2 5 0 6 1 8 4 4 3 5 2 5 4 4 0 4 4 2
 8 3 0 0 7 0 4 3 7 2 2 2 1 0 7 6 2 3 7 2 4 6 4 4 6 1 2 0 1 1 8 0 5 1 1 2 1
 4 0 6 3 5 2 5 1 2 5 2 4 3 5 2 0 0 4 6 0 4 0 8 6 4 4 6 4 6 8 0 0 3 7 1 3 5
 0 8 3 5 0 0 3 5 4 5 4 4 5 7 0 7 1 0 7 4 5 5 1 5 6 7 5 7 7 0 2 6 4 5 8 0 4
 3 2 7 3 4 7 3 5 4 3 4 5 4 7 3 1 0 7 0 2 4 0 4 4 4 7 1 0 7 4 3 6 0 1 1 5 2
 2 3 7 0 6 7 2 0 0 3 4 0 

## TF-IDF预处理

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
#使用tf-idf把文本转为向量
tv = TfidfVectorizer(stop_words=stopwords, max_features=5000, lowercase = False)
tv.fit(x_train_fenci)
#模型训练
classifier.fit(tv.transform(fenci(x_train)), y_train)
#利用训练好的模型测试
classifier.score(tv.transform(fenci(x_test)), y_test)


D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.6955074875207987

In [28]:
from sklearn.metrics import classification_report
predict=classifier.predict(tv.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.60      0.61      0.61        57
           1       0.83      0.61      0.71        88
           2       0.72      0.83      0.77        64
           3       0.70      0.69      0.69        64
           4       0.77      0.79      0.78        72
           5       0.75      0.81      0.78        70
           6       0.54      0.77      0.63        52
           7       0.80      0.45      0.58        95
           8       0.56      0.90      0.69        39

    accuracy                           0.70       601
   macro avg       0.70      0.72      0.69       601
weighted avg       0.72      0.70      0.69       601



## N-gram 模型

In [18]:
tv_2gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,2),lowercase = False)
tv_2gram.fit(x_train_fenci)
#训练模型
clf_2gram = MultinomialNB()
clf_2gram.fit(tv_2gram.transform(fenci(x_train)), y_train)
#预测
clf_2gram.score(tv_2gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.6905158069883528

In [27]:
from sklearn.metrics import classification_report
predict=clf_2gram.predict(tv_2gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.62      0.63      0.63        57
           1       0.83      0.68      0.74        80
           2       0.72      0.79      0.75        67
           3       0.68      0.69      0.69        62
           4       0.76      0.80      0.78        70
           5       0.74      0.84      0.78        67
           6       0.55      0.77      0.65        53
           7       0.76      0.39      0.52       104
           8       0.56      0.85      0.67        41

    accuracy                           0.69       601
   macro avg       0.69      0.72      0.69       601
weighted avg       0.71      0.69      0.68       601



## 3-gram

In [24]:
tv_3gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,3),lowercase = False)
tv_3gram.fit(x_train_fenci)
#训练模型
clf_3gram = MultinomialNB()
clf_3gram.fit(tv_3gram.transform(fenci(x_train)), y_train)
#预测
clf_3gram.score(tv_3gram.transform(fenci(x_test)), y_test)

D:\Software\Anaconda\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


0.6888519134775375

In [26]:
from sklearn.metrics import classification_report
predict=clf_3gram.predict(tv_3gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率\n",classification_report(predict, y_test))

每个类别的精确率与召回率
               precision    recall  f1-score   support

           0       0.64      0.66      0.65        56
           1       0.80      0.65      0.72        80
           2       0.73      0.79      0.76        68
           3       0.67      0.65      0.66        65
           4       0.76      0.80      0.78        70
           5       0.74      0.82      0.78        68
           6       0.54      0.75      0.63        53
           7       0.76      0.41      0.54        99
           8       0.57      0.86      0.69        42

    accuracy                           0.69       601
   macro avg       0.69      0.71      0.69       601
weighted avg       0.70      0.69      0.68       601

